In [1]:
import pandas as pd
import glob

In [2]:
Path = r'E:\_Projects\Schedule Stability\\'
Completion_Path = r'E:\_Projects\completions\reports\\'

In [3]:
year_month = int(input("Enter Report Date (YYYYMM) : "))

Enter Report Date (YYYYMM) :  202508


In [4]:
print('Reading input files................!')
Product_Range =pd.read_excel(Path+'Product Range.xlsx')

try: 
    Glob_Doc = glob.glob(Path+'Input\\Monthly\\Global DOC Atleos?????????.xlsb')
    print('Global Doc File : ',Glob_Doc)
    for Glob_Doc in Glob_Doc:
        GLOB_DOC =pd.read_excel(Glob_Doc,sheet_name='Orders',engine='pyxlsb')
except:
    print('Global file Not found')

COMP = pd.read_excel(Completion_Path+'Completion.xlsx',sheet_name='Data')
Plan = pd.read_csv(Path+'Input\\Monthly\\Plan.csv')

print('********* All input files Loaded *********')

Reading input files................!
Global Doc File :  ['C:\\Vasanthan\\Report\\Schedule Stability\\Schedule stability Monthly\\Automation\\\\Input\\Global DOC Atleos 20250801.xlsb']
********* All input files Loaded *********


### ############## Month & Year Extract

In [5]:
x = year_month%100

Month =''

Year = int(str(year_month)[:4])



if x==1:
    Month='Jan'
elif x==2:
    Month='Feb'
elif x==3:
    Month='Mar'
elif x==4:
    Month='Apr'
elif x==5:
    Month='May'
elif x==6:
    Month='Jun'
elif x==7:
    Month='Jul'
elif x==8:
    Month='Aug'
elif x==9:
    Month='Sep'
elif x==10:
    Month='Oct'
elif x==11:
    Month='Nov'
elif x==12:
    Month='Dec'
else:
    print("Enter validate Year and Month")

print('Year :',Year)
print('Month : ',Month)



Year : 2025
Month :  Aug


### ############### Plan Extraction

In [6]:
Plan = Plan.loc[(Plan['Plant SSD Mon']==year_month) & (Plan['PID Type']=='M')]

Plan = Plan[['PID','PID Type','Plant','Products','Supply Qty']]



CHE_ATM = Plan[(Plan['Plant']=='CHE') & (Plan['Products']=='ATM')]
BUD_ATM = Plan[(Plan['Plant']=='BUD') & (Plan['Products']=='ATM')]


print(CHE_ATM['Supply Qty'].sum())
print(BUD_ATM['Supply Qty'].sum())


Plan = pd.concat([CHE_ATM,BUD_ATM],axis=0)

Plan = Plan[['PID','Plant','Supply Qty']]


Plan.rename(columns={'PID':'Product ID'},inplace=True)


Plan.rename(columns={'Supply Qty':'Plan Qty'},inplace=True)


Plan = Plan.groupby(['Plant','Product ID']).agg({'Plan Qty':'sum'})


Plan.to_csv(Path+'\\Output\\ref\\Plan.csv')



5031
1405


### ########### Completion

In [7]:

########################################## Completion ############################

COMP = COMP.loc[(COMP['ship_date_mth']==year_month) & (COMP['item'].str.contains('MC')) & (COMP['site'].astype(str).isin(['BUD','CHS','CHE'])) & (COMP['prod_grp_wb'].str.contains('ATM'))]
COMP.loc[COMP['site']=='CHS','site']='CHE'

COMP = COMP[['site','item','prod_grp_wb','qty']]


COMP = COMP.groupby(['site','item','prod_grp_wb']).agg({'qty':'sum'})

COMP.to_csv(Path+'\\Output\\ref\\Completion.csv')


### ###################### Plan Vs Completion

In [8]:
Plan = pd.read_csv(Path+'\\Output\\ref\\Plan.csv')
COMP = pd.read_csv(Path+'\\Output\\ref\\Completion.csv')

Plan['Concat'] = Plan['Plant']+Plan['Product ID']
COMP['Concat'] = COMP['site']+COMP['item']


Plan = pd.merge(Plan, COMP,on='Concat',how='outer')

Plan = Plan.rename(columns=lambda x: x.replace('qty', 'Actual Completions'))

Plan.loc[Plan['Plant'].isnull(),'Plant'] = Plan['site']
Plan.loc[Plan['Product ID'].isnull(),'Product ID'] = Plan['item']

Plan['Actual Completions'] = Plan['Actual Completions'].fillna(0)
Plan['Plan Qty'] = Plan['Plan Qty'].fillna(0)
Plan = Plan[['Plant','Product ID','Plan Qty','Actual Completions']]

Plan['Concat'] = Plan['Plant']+Plan['Product ID']



Plan.loc[(Plan['Plan Qty']-Plan['Actual Completions']<=0),'Planned Completions']=Plan['Plan Qty']
Plan.loc[(Plan['Plan Qty']-Plan['Actual Completions']>0),'Planned Completions']=Plan['Actual Completions']

Plan.loc[(Plan['Plan Qty']-Plan['Actual Completions'])==0,'Result']='CLEAN'
Plan.loc[(Plan['Plan Qty']-Plan['Actual Completions'])<0,'Result']='UNPLANNED'
Plan.loc[(Plan['Plan Qty']-Plan['Actual Completions'])>0,'Result']='MISS'

Plan.loc[(Plan['Plan Qty']-Plan['Actual Completions'])>0,'Miss Qty']=(Plan['Plan Qty']-Plan['Actual Completions'])
Plan.loc[(Plan['Plan Qty']-Plan['Actual Completions'])<0,'Unplanned Qty']=(Plan['Actual Completions'] - Plan['Plan Qty'])


Plan.loc[(Plan['Plan Qty']-Plan['Actual Completions'])<=0,'Clean Qty']=Plan['Plan Qty']

Plan['Class'] = Plan['Product ID'].astype(str).str[:4]



Plan['Class'] = Plan['Class'].astype(str)
Product_Range['Class'] = Product_Range['Class'].astype(str)


Plan = pd.merge(Plan, Product_Range,on='Class',how='left')

### ################## UPDATING CUSTOMER NAME

In [9]:
GLOB_DOC = GLOB_DOC[['PID','Whse','Customer/End Customer']]

GLOB_DOC = GLOB_DOC[(GLOB_DOC['Customer/End Customer']!='NCR INTERCOMPANY') & GLOB_DOC['Customer/End Customer'].notna()]


GLOB_DOC['Whse'] = GLOB_DOC['Whse'].replace('ENN','BUD')

GLOB_DOC['Concat'] = GLOB_DOC['Whse']+GLOB_DOC['PID']

GLOB_DOC = GLOB_DOC.drop_duplicates(subset=['PID','Whse'], keep='first')

Plan = pd.merge(Plan, GLOB_DOC,on='Concat',how='left')

Plan = Plan.rename(columns=lambda x: x.replace('Customer/End Customer', 'Customer'))

Plan['Product ID2']=''

Plan['Month'] = Month
Plan['Year'] = Year


Schedule_Stability = Plan[['Plant','Month','Year','Product Range','Prod Range 2','Class','Product ID2','Product ID','Customer','Plan Qty','Actual Completions','Planned Completions','Result','Miss Qty','Unplanned Qty','Clean Qty']]

Schedule_Stability['Customer'] = Schedule_Stability['Customer'].fillna('Other-')

Schedule_Stability.to_csv(Path+'\\Output\\Schedule Stability Monthly.csv',index=False)



C:\Users\vv185114\AppData\Local\Temp\ipykernel_28704\3354182898.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Schedule_Stability['Customer'] = Schedule_Stability['Customer'].fillna('Other-')


In [13]:
#############################################

BUD_Plan = Schedule_Stability[Schedule_Stability['Plant']=='BUD']
CHE_Plan_ATM = Schedule_Stability[(Schedule_Stability['Plant'].isin(['CHE','CHS'])) &(Schedule_Stability['Product Range']=='ATM')]


BUD_Plan_Qty = BUD_Plan['Plan Qty'].sum()
BUD_Actual_Completions = BUD_Plan['Actual Completions'].sum()

CHE_Plan_ATM_Qty = CHE_Plan_ATM['Plan Qty'].sum()
CHE_Actual_ATM_Completions = CHE_Plan_ATM['Actual Completions'].sum()


print('**********************************************')
print('BUD Plan       : ', BUD_Plan_Qty)
print('CHE Plan ATM   : ', CHE_Plan_ATM_Qty)

print('**********************************************')
print('BUD Completion     : ',BUD_Actual_Completions)
print('CHE ATM Completion : ',CHE_Actual_ATM_Completions)

print('**REPOTE AUTOMATION COMPLETED**')

**********************************************
BUD Plan       :  1405.0
CHE Plan ATM   :  5031.0
**********************************************
BUD Completion     :  1416.0
CHE ATM Completion :  5530.0
**REPOTE AUTOMATION COMPLETED**
